In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S')

In [3]:
import sys

sys.path.insert(0, '../..')

## Load Dataset

In [4]:
from entity_embed.benchmarks import WalmartAmazonStructuredBenchmark

benchmark = WalmartAmazonStructuredBenchmark(data_dir_path="../data/")
benchmark

17:28:11 INFO:Extracting Walmart-Amazon-Structured...
17:28:11 INFO:Reading Walmart-Amazon-Structured row_dict...
17:28:11 INFO:Reading Walmart-Amazon-Structured train.csv...
17:28:11 INFO:Reading Walmart-Amazon-Structured valid.csv...
17:28:11 INFO:Reading Walmart-Amazon-Structured test.csv...


<WalmartAmazonStructuredBenchmark> from http://pages.cs.wisc.edu/~anhai/data1/deepmatcher_data/Structured/Walmart-Amazon/walmart_amazon_exp_data.zip

## Preprocess

In [5]:
attr_list = ['title', 'category', 'brand', 'modelno', 'price']

In [6]:
from tqdm.auto import tqdm
import unidecode

def clean_str(s):
    return unidecode.unidecode(s).lower().strip()

for row in tqdm(benchmark.row_dict.values()):
    for attr in attr_list:
        row[attr] = clean_str(row[attr])

  0%|          | 0/24628 [00:00<?, ?it/s]

## Init Data Module

In [7]:
import torch
import numpy as np

random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)

In [8]:
alphabet = list('0123456789abcdefghijklmnopqrstuvwxyz!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ ')

In [9]:
attr_config_dict = {
    'title': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'title_semantic': {
        'source_attr': 'title',
        'field_type': "SEMANTIC_MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'vocab': "fasttext.en.300d",
    },
    'category': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'category_semantic': {
        'source_attr': 'category',
        'field_type': "SEMANTIC_MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'vocab': "fasttext.en.300d",
    },
    'brand': {
        'field_type': "STRING",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'modelno': {
        'field_type': "STRING",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'price': {
        'field_type': "STRING",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
}

In [10]:
from entity_embed import AttrConfigDictParser

row_numericalizer = AttrConfigDictParser.from_dict(attr_config_dict, row_list=benchmark.row_dict.values())
row_numericalizer.attr_config_dict

17:28:11 INFO:For attr=title, computing actual max_str_len
17:28:11 INFO:actual_max_str_len=23 must be even to enable NN pooling. Updating to 24
17:28:11 INFO:For attr=title, using actual_max_str_len=24
17:28:11 INFO:Loading vectors from .vector_cache/wiki.en.vec.pt
17:28:15 INFO:For attr=category, computing actual max_str_len
17:28:15 INFO:actual_max_str_len=15 must be even to enable NN pooling. Updating to 16
17:28:15 INFO:For attr=category, using actual_max_str_len=16
17:28:15 INFO:Loading vectors from .vector_cache/wiki.en.vec.pt
17:28:19 INFO:For attr=brand, computing actual max_str_len
17:28:19 INFO:For attr=brand, using actual_max_str_len=46
17:28:19 INFO:For attr=modelno, computing actual max_str_len
17:28:19 INFO:For attr=modelno, using actual_max_str_len=48
17:28:19 INFO:For attr=price, computing actual max_str_len
17:28:19 INFO:For attr=price, using actual_max_str_len=8


{'title': AttrConfig(source_attr='title', field_type=<FieldType.MULTITOKEN: 'multitoken'>, tokenizer='entity_embed.data_utils.numericalizer.default_tokenizer', alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', ' '], max_str_len=24, vocab=None, n_channels=8, embed_dropout_p=0.2, use_attention=True, use_mask=True),
 'title_semantic': AttrConfig(source_attr='title', field_type=<FieldType.SEMANTIC_MULTITOKEN: 'semantic_multitoken'>, tokenizer='entity_embed.data_utils.numericalizer.default_tokenizer', alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', '"

In [11]:
datamodule = benchmark.build_datamodule(
    row_numericalizer=row_numericalizer,
    batch_size=32,
    eval_batch_size=128,
    random_seed=random_seed
)

## Training

In [12]:
from entity_embed import Matcher

model = Matcher(
    row_numericalizer=row_numericalizer,
)

In [13]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

max_epochs = 100
early_stop_callback = EarlyStoppingMinEpochs(
   monitor='valid_f1_at_0.5',
   min_delta=0.00,
   patience=10,
   verbose=True,
   mode='max'
)
tb_save_dir = '../tb_logs'
tb_name = f'matcher-{benchmark.dataset_name}'
trainer = pl.Trainer(
    gpus=1,
    max_epochs=max_epochs,
    check_val_every_n_epoch=1,
    callbacks=[early_stop_callback],
    logger=TensorBoardLogger(tb_save_dir, name=tb_name),
    reload_dataloaders_every_epoch=True
)

GPU available: True, used: True
17:28:19 INFO:GPU available: True, used: True
TPU available: None, using: 0 TPU cores
17:28:19 INFO:TPU available: None, using: 0 TPU cores


In [14]:
trainer.fit(model, datamodule)

17:28:19 INFO:Train pair count: 576
17:28:19 INFO:Valid pair count: 193
17:28:19 INFO:Test pair count: 193
17:28:19 INFO:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type       | Params
-------------------------------------------
0 | blocker_net | BlockerNet | 22.4 M
1 | losser      | SupConLoss | 0     
-------------------------------------------
13.4 M    Trainable params
9.0 M     Non-trainable params
22.4 M    Total params
89.721    Total estimated model params size (MB)
17:29:54 INFO:
  | Name        | Type       | Params
-------------------------------------------
0 | blocker_net | BlockerNet | 22.4 M
1 | losser      | SupConLoss | 0     
-------------------------------------------
13.4 M    Trainable params
9.0 M     Non-trainable params
22.4 M    Total params
89.721    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

1

In [15]:
model.blocker_net.get_signature_weights()

{'title': 0.26879751682281494,
 'title_semantic': 0.29293301701545715,
 'category': 0.04999953508377075,
 'category_semantic': 0.03403683006763458,
 'brand': 0.18322932720184326,
 'modelno': 0.12046590447425842,
 'price': 0.0505378283560276}

In [16]:
from entity_embed import validate_best

validate_best(trainer)

17:35:47 INFO:Validating best model...


{'valid_f1_at_0.3': 0.3634651600753296,
 'valid_f1_at_0.5': 0.6701754385964912,
 'valid_f1_at_0.7': 0.8592233009708738,
 'valid_f1_at_0.9': 0.5860805860805861,
 'valid_pair_entity_ratio_at_0.3': 2.29287598944591,
 'valid_pair_entity_ratio_at_0.5': 0.9947229551451188,
 'valid_pair_entity_ratio_at_0.7': 0.5778364116094987,
 'valid_pair_entity_ratio_at_0.9': 0.21108179419525067,
 'valid_precision_at_0.3': 0.22209436133486765,
 'valid_precision_at_0.5': 0.506631299734748,
 'valid_precision_at_0.7': 0.8082191780821918,
 'valid_precision_at_0.9': 1.0,
 'valid_recall_at_0.3': 1.0,
 'valid_recall_at_0.5': 0.9896373056994818,
 'valid_recall_at_0.7': 0.917098445595855,
 'valid_recall_at_0.9': 0.41450777202072536}

## Testing

In [17]:
trainer.test(ckpt_path='best', verbose=False)

17:35:49 INFO:Train pair count: 576
17:35:49 INFO:Valid pair count: 193
17:35:49 INFO:Test pair count: 193
17:35:49 INFO:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

[{'test_f1_at_0.3': 0.4063157894736842,
  'test_f1_at_0.5': 0.7258979206049149,
  'test_f1_at_0.7': 0.8866498740554156,
  'test_f1_at_0.9': 0.5735294117647058,
  'test_pair_entity_ratio_at_0.3': 1.9713541666666667,
  'test_pair_entity_ratio_at_0.5': 0.875,
  'test_pair_entity_ratio_at_0.7': 0.53125,
  'test_pair_entity_ratio_at_0.9': 0.20572916666666666,
  'test_precision_at_0.3': 0.25495376486129456,
  'test_precision_at_0.5': 0.5714285714285714,
  'test_precision_at_0.7': 0.8627450980392157,
  'test_precision_at_0.9': 0.9873417721518988,
  'test_recall_at_0.3': 1.0,
  'test_recall_at_0.5': 0.9948186528497409,
  'test_recall_at_0.7': 0.9119170984455959,
  'test_recall_at_0.9': 0.40414507772020725}]